# Adding noise to the EAGLE simulations
This goal of this notebook is to explore adding noise to the EAGLE simulation to make mock observations.  The main part of this notebook doesn't even use the simulations, it just explores the sources of noise as added to a pretend data set (a circle on a blank background).

We will go through the different types of noise so that it is clear which one dominates the signal.  In particular, we want to explore when read out noise trades out with the sky background as the dominant noise source.

### Sources of Poisson noise:
The shot noise, dark current noise, and sky background noise are all Poisson processes.  Therefore can draw from a Poisson or Gaussian (large number statistics) distribution with a sigma that is the square root of the mean value for each background source to draw random values for the noise.

> For small photon counts, photon noise is generally dominated by other signal-independent sources of noise, and
for larger counts, the central limit theorem ensures that the Poisson distribution approaches a Gaussian.

> The ratio of signal to photon noise grows with the square root
of the number of photons captured, √ λt. <br>
( https://people.csail.mit.edu/hasinoff/pubs/hasinoff-photon-2012-preprint.pdf )

Poission_noise = sigma = SQRT(meanvalue_Poisson)

By the Central Limit Theorem, the Poisson distribution can be approximated as a Gaussian ( http://www.socr.ucla.edu/Applets.dir/NormalApprox2PoissonApplet.html ): <br>
if X $\sim$ Poisson($\lambda$) $\Rightarrow$ X $\approx N(\mu = \lambda,\sigma = \sqrt\lambda )$ 


e.g. D_noise = SQRT ( D )

### Read out noise:
The read out noise is a Gaussian process, therefore can draw from a Gaussian distribution with a sigma that is the read out noise value given.  

> CCD manufacturers measure and report CCD noise as a number of electrons RMS (Root Mean Square).  You’ll typically see it presented like this, 15eˉ RMS, meaning that with this CCD, you should expect to see about 15 electrons of noise per pixel.  More precisely, 15eˉ RMS is the standard deviation around the mean pixel value. <br>
( http://www.qsimaging.com/ccd_noise.html )

It seems that we don't actually know the mean value of the read out noise, so we should center the read out noise at zero (since we can subtract the mean value off in the images).

Gaussian_noise = sigma = R

### Combining noise:
We then add the noise in quadrature (error propagation):

Noise_total = SQRT ( Poisson_noise ^ 2 + Gaussian_noise ^2 + ... ) = SQRT ( (meanvalue_Poisson) + R^2 + ...)

For a couple references on error propagation, check out:
http://www.mso.anu.edu.au/pfrancis/ObsTech/Stats2.pdf and https://www.deanza.edu/faculty/marshburnthomas/pdf/ErrorPropagation.pdf.


In [1]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

import get_halpha_SB

%matplotlib inline

In [2]:
%run 'load_data.ipynb'

First, let's declare some variables from the Dragonfly Telephoto array.

We will also declare a background value, which was taken from the Gemini Sky Background spectrum.  
*Verify that this background is correct*

In [3]:
# Dragonfly info
area_lens = np.pi*(14.3/2)**2 * 48.                     # cm^2, 48 * 14.3 cm diameter lenses
pix_size = 2.8                                          # arcsec
ang_size_pixel  = (pix_size * (1./206265.))**2          # rad^2, the pixel size of the CCD
tau_l = 0.85                                            # transmittance of the Dragonfly lens
tau_f = 1.                                              # transmittance of the Halpha filter -- assumed for now
#B = getBackground(656.3,657.3,machine)                  # *u.photon/u.second/u.arcsec**2/u.m**2  ****already multiplied by the bandwidth***
B = 0.560633
D = 0.04       # dark current (electrons / s) 
QE= 0.48 # current cameras

In [4]:
#debugging = True

Adding the sky background (and the Poisson noise background from the data as well, in one go).

1. Grab sky background 
2. Multiply sky background by efficiency factors, by the aperture area, and the by square angular pixel size
3. Multiply sky background by the exposure time
4. Multiply sky background by the number of pixels in one bin
5. Make an array the shape of the data, to put the sky background and Poisson noise into
6. Draw from a random gaussian distribution of the sqrt of the background (and the data) for each element of the array
7. Return the array of sky background and Poisson noise, and the value of the sky background for a binned pixel in the exposure time

In [5]:
def add_skybackground(data,B_sky,exptime,numpixel):
    'Background from stuff in space'
    B_sky_inexptime = B_sky*exptime
    B_sky_total     = B_sky_inexptime*numpixel    
    B_sky_array = np.zeros((data.shape[0],data.shape[1]))
    if debugging:
        print "DEBUGGING: the shape of B_sky_array is: "
        print B_sky_array.shape
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            B_sky_array[x][y]=np.random.normal(0,np.sqrt(B_sky_total+data[x][y])) 
#            B_sky_array[x][y]=np.random.poisson(B_sky_total)

    if debugging:
        print "DEBUGGING: the mean total background signal, B_sky_total [electrons], is: %s"\
                %B_sky_total
        print "DEBUGGING: the total background noisy signal [electrons] ranges from: %s to %s"\
                %(np.min(B_sky_array),np.max(B_sky_array))
 
    return B_sky_array

Adding the dark current. 

In [12]:
def add_darkcurrent(data,exptime,numpixel):
    'DarkCurrent'
    noise_from_detector = 0.0
    D_total = D*exptime*numpixel
    D_array_total = np.zeros((data.shape[0],data.shape[1]))
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            D_array_total[x][y]=np.random.normal(D_total,np.sqrt(D_total)) 
    if debugging:
        print "DEBUGGING: the total dark current [electrons] is: %s"%(D_total)
        print "DEBUGGING: the total dark current noisy signal [electrons] ranges from: %s to %s"\
                %(np.min(D_array_total),np.max(D_array_total))
        
    return D_array_total

Adding the read out noise.  *Note to self: I don't know why the sigma was B_sky for the read out noise in the previous script... i'm guessing for testing but it is very confusing to me*

1. If the number of exposures is not provided, assume the exposures are an hour long, so get the number of times data read out by dividing exposure time by 3600 secounds
2. The read out noise is Gaussian distributed with sigma R (10e- for Dragonfly) so return a random value draw from a Gaussian distribution with sigma R

This algorithm for applying the readout noise assumes that each 2.8 arcsecond pixel (i.e. inherent Dragonfly pixel, not binned) is assigned a (random) readout noise.  

Since we will end up binning the data over some number of pixels, "numpixel", we build an array the size of the binned array (smaller than the inherent Dragonfly array), and in each index of the array, we store the SUM of the readout noise from each of the pixels and from each exposure.  We store the SUM instead of the AVERAGE because we won't be able to calculate the average readout noise independantly in the actual data (unless we do on-chip binning).  (So, right now I am assuming off-chip binning).

The binned read out noise is stored in "R_array".

The squared of "R_array" is stored in "R_squared_array", in prep for adding the noise in quadrature.  *Note to self:  Wait, why am I doing this?*



In [10]:
def add_readoutnoise(data,R,numpixel,expnum,exptime):
    'ReadOut Noise'
    numexp = 1
    numlens = 48    
    R_squared = R**2
    if expnum is None:
        numexp = exptime/3600. # hour long exposures
        if debugging:
            print "DEBUGGING: No expnum provided, assuming hour long exposures."
    else:
        numexp = expnum
    if debugging:
        print "DEBUGGING: The number of exposures is: %s" % numexp
        print "DEBUGGING: Drawing %s values (numpixel=%s * numexp=%s * numlens=%s) per binned pixel and summing them." \
                %((int(numpixel)*round(numexp)*numlens),int(numpixel),round(numexp),numlens)
    
    R_squared_array = np.zeros((data.shape[0],data.shape[1]))
    R_array = np.zeros((data.shape[0],data.shape[1]))
    for x in range(data.shape[0]):
        print "VERBOSE:  At line %s out of %s" % (x,data.shape[0])
        #if data.shape[0]>1000:
        #    if (x % 100)==0:
        #        print "VERBOSE:  At line %s out of %s" % (x,data.shape[0])
        #elif data.shape[0]>100:
        #    if (x % 50)==0:
        #        print "VERBOSE: At line %s out of %s" % (x,data.shape[0])
        for y in range(data.shape[1]):
            if data.shape[1]>1000:
                if (y % 100)==0:
                    print "VERBOSE:  At column %s out of %s" % (y,data.shape[0])
            R_array[x][y]=np.sum(np.random.normal(0,R,int(numpixel)*round(numexp)*numlens))
            R_squared_array[x][y]=(R_array[x][y])**2 # centered at 0, st dev of R, return numpixel*numexp values
    
    if debugging:
        print "DEBUGGING: Standard deviation of R array: %s " % np.std(R_array)
        print "DEBUGGING: the R_squared value is: %s, so in %s exposures [per pixel], will have R_squared of: %s"\
                %(R_squared,numexp,R_squared * round(numexp))
        print "DEBUGGING: the total R_squared value [electrons] multiplying by numpix read out is: %s"\
                %((R_squared * round(numexp) * numpixel))
        print "DEBUGGING: the max and min R_squared values are: %s and %s" \
                %(np.max(R_squared_array),np.min(R_squared_array))
        print "DEBUGGING: the max and min R values are: %s and %s" \
                %(np.max(R_array),np.min(R_array))
        
    return R_array

In [46]:
def addnoise(data,resolution,log = True,R=None,expnum=None, exptime=10**3*3600.,CMOS=False, debugging=True):
    """
    This function adds noise to simulated data (such as the EAGLE simulation) to create mock observations.
    The EAGLE data is logged though, so the log = True for the EAGLE stuff
    """
    binpix_size = resolution # arcsec
    numpixel = round((binpix_size/pix_size)**2)
    
    if debugging:
        print "DEBUGGING: the binpix_size (resolution) is %s" %binpix_size
        print "DEBUGGING: the pixel size (inherent) is %s" %pix_size
    
    if CMOS:
        print "VERBOSE: Using new CMOS cameras... (QE = 0.70, R = 2.)"
        QE = 0.70                                           # quantum efficiency of the CMOS detector
    else:
        print "VERBOSE: Using old cameras... (QE = 0.48, R = 10.)"
        QE = 0.48    
    
    if R is None:
        if CMOS:
            R = 2.                               # read noise (electrons)
        else:
            R = 10.                              
    
    if debugging:
        print "DEBUGGING: R is : %s" % R
        print "DEBUGGING: the number of pixels is %s" % numpixel
    
    if log:
        if debugging:
            print "DEBUGGING: raise the data by 10** since was logged data before..."
        data = 10**data
    
    if debugging:
        print "DEBUGGING: the shape of the input data is:"
        print data.shape
    
    'total signal incident in exposure time'
    totsignal = data * exptime # ( photons / cm^2 /sr )
    'total signal detected (accounting for system efficiency)'
    detsignal = totsignal * QE * tau_l * tau_f * area_lens * ang_size_pixel * numpixel
    
    if debugging:
        print "DEBUGGING: the total object signal [electrons] detected ranges from: %s to %s"%(np.min(detsignal),np.max(detsignal))
        #print "DEBUGGING: an example of the object signal [electrons] is: %s"%detsignal[0]
        print "DEBUGGING: the shape of the detsignal is:"
        print detsignal.shape
    
    print "Adding sky background noise and shot noise."
    'background sky signal detected [B]=ph/s/arcsec^2/m^2, [B_sky]=ph/s (in a pixel)'
    B_sky = B * QE * tau_l * tau_f * area_lens*(1/100.)**2 * pix_size**2
    if debugging:
        print "DEBUGGING: the background in the bandwidth is: %s"%B
        print "DEBUGGING: the background signal, B_sky [ph/s (in a pixel)], is: %s"%B_sky

    B_sky_array = add_skybackground(detsignal,B_sky,exptime,numpixel)
    noiseadded_signal = detsignal + B_sky_array + B_sky*exptime*numpixel  ## last term is constant (mean sky background - don't really need to add in...)
    
    print "Adding read out noise to the signal."
    R_array = add_readoutnoise(detsignal,R,numpixel,expnum,exptime)
    noise_from_detector = R_array

    print "Adding dark current to the signal."
    D_array = add_darkcurrent(detsignal,exptime,numpixel)
    noise_from_detector = noise_from_detector + D_array
        
    noiseadded_signal = noiseadded_signal + noise_from_detector
    
    return noiseadded_signal,B_sky_array,R_array

In [74]:
### EXAMPLE ###
## exposure time
#exptime = 60.*60.*10**5  # seconds  (10^5 hours)
## where is the data
#machine='coho'
## what distance and resolution do you want
#distance = '50Mpc'; resolution = 100
#data_tuple = loaddata(machine=machine,resolution=resolution,distance=distance)
## or do you want the raw data
#data_tuple = loaddata(machine=machine)
#data = data_tuple[0]